In [1]:
import csv
import numpy as np

MJUPITER = 1.898e27
MEARTH = 5.972e24
RJUPITER = 69.911e6
REARTH = 6371e3

In [2]:
class System:
    def __init__(self, sysid=None):
        self.sysid = sysid

In [3]:
my_file = 'C:/Users/djhoo/Documents/maiasaurus-master/catalogues/oviraptor_crossmatch_catalog.csv'  #reading in file

raw_data = []
with open(my_file) as infile:
    reader = csv.reader(infile)
    
    for row in reader:
        raw_data.append(row)
        
KEYS = raw_data[0]
VALUES = raw_data[1:]

def get_key(keyname, keys = KEYS, values = VALUES):
    i = keys.index(keyname)
    arr = []
    for row in values:
        arr.append(row[i])
    return np.array(arr)

d = {}
d['starname'] = get_key('hostname')
d['rp'] = get_key('pl_rade')
d['rerr1'] = get_key('pl_radeerr1')
d['rerr2'] = get_key('pl_radeerr2')
d['mp'] = get_key('pl_bmasse')
d['merr1'] = get_key('pl_bmasseerr1')
d['merr2'] = get_key('pl_bmasseerr2')
d['rs'] = get_key('st_rad')
d['ms'] = get_key('st_mass')
d['p'] = get_key('pl_orbper')
d['disc_method'] = get_key('discoverymethod')
d['temp'] = get_key('st_teff')
d['mag'] = get_key('sy_vmag')
d['ref'] = get_key('st_refname')
d['instrument'] = get_key('disc_facility')

print('Number of oviraptor planets: %d' %len(d['starname']))

Number of oviraptor planets: 4097


In [4]:
def convert_masses(data):
    masses = np.zeros(len(data))
    for i in range(len(masses)):
        if data[i] != '':
            masses[i] = float(data[i])*MJUPITER/MEARTH
    return masses

def convert_radii(data):
    radii = np.zeros(len(data))
    for i in range(len(radii)):
        if data[i] != '':
            radii[i] = float(data[i])*RJUPITER/REARTH
    return radii

In [5]:
my_file = 'C:/Users/djhoo/Documents/maiasaurus-master/catalogues/exoarchive_ipac_confirmed_200507.csv'
raw_data = []
with open(my_file) as infile:
    reader = csv.reader(infile)
    
    for row in reader:
        raw_data.append(row)
        
KEYS = raw_data[77]
VALUES = raw_data[78:]

def get_key(keyname, keys = KEYS, values = VALUES):
    i = keys.index(keyname)
    arr = []
    for row in values:
        arr.append(row[i])
    return arr

names = get_key('pl_hostname')     
rs = get_key('st_rad')
ms = get_key('st_mass')

print('Number of Exoarchive planets: %d' %len(names))

'''
if exoarchive stellar mass/radius is something and corresponding oviraptor stellar mass/radius is nothing:
replace d['ms'] or d['rs'] with corresponding value
'''

for i in range(len(names)):
    if ms[i] != '' and d['ms'][d['starname'] == names[i]].tolist() == ['']*len(d['ms'][d['starname'] == names[i]]):
        for j in range(len(d['ms'])):
            if d['starname'][j] == names[i]:
                d['ms'][j] = ms[i]
    if rs[i] != '' and d['rs'][d['starname'] == names[i]].tolist() == ['']*len(d['rs'][d['starname'] == names[i]]):
        for j in range(len(d['rs'])):
            if d['starname'][j] == names[i]:
                d['rs'][j] = rs[i]

Number of Exoarchive planets: 4152


In [6]:
def check_uniformity(detection_methods):
    default_disc = detection_methods[0]
    status = True
    for i in range(len(detection_methods)):
        if detection_methods[i] != default_disc:
            status = False
    return status

d['system_disc_method'] = []

for i in range(len(d['starname'])):
    if check_uniformity(d['disc_method'][d['starname']==d['starname'][i]]) == False:
        d['system_disc_method'].append('Mixed')
    else:
        d['system_disc_method'].append(d['disc_method'][i])

In [7]:
for i in range(len(d['starname'])):
    radii = d['rs'][d['starname'] == d['starname'][i]]               #replicating values of stellar radius for 
    if len(radii[radii != '']) > 0:                                  #planets withint the same system
        true_radius = radii[radii != ''][0]
        d['rs'][i] = true_radius
        
    mass = d['ms'][d['starname'] == d['starname'][i]]                #doing basically the same thing but for stellar mass
    if len(mass[mass != '']) > 0:
        true_mass = mass[mass != ''][0]
        d['ms'][i] = true_mass

In [8]:
gaia_stars = {}
gaia_stars['stars'] = ['HD 136352', 'HD 141399', 'HD 160691', 'HD 20781', 'HD 20794', 'HD 27984', 'HD 31527', \
                      'HD 40307', 'tau Cet']
gaia_stars['radii'] = [1.0041078, 1.3874089, 1.3901234, 0.8336087, 0.83483344, 0.89176697, 1.0736936, 1.3310363, \
                      0.793]  

#all of these radii are taken from the Gaia archive
#except for tau Cet, which was taken from Dietrich and Apai (2020)

for i in range(len(d['starname'])):
    if d['starname'][i] in gaia_stars['stars']:
        d['rs'][i] = gaia_stars['radii'][gaia_stars['stars'] == d['starname'][i]]

In [9]:
'''
The data entries are the following:

- Starname - name of host star
- Rp - radius of exoplanet (in Earth radii)
- Mp - mass of exoplanet (in Earth masses)
- P - period of exoplanet (in days)
- Rs - radius of host star (in solar radii)
- Ms - mass of host star (in solar masses)
- Detection Method - detection method (Radial Velocity, Transit, or Mixed)
- System Detection Method - the detection method of a system as a whole
- Temp - temperature of the star (in Kelvin)
- Mag - visual magnitude of the star
- Instrument - discovery instrument of the star
- Ref - reference paper
'''

output_heading = ['Starname', 'Rp', 'R_upper', 'R_lower', 'Mp', 'M_upper', 'M_lower', 'P', 'Rs', 'Ms', \
                  'Detection Method', 'System Detection Method', 'Temp', 'Mag', 'Instrument', 'Ref']
myData = [output_heading]
for i in range(len(d['starname'])):
    arr = []
    arr.append(d['starname'][i])
    arr.append(d['rp'][i])
    arr.append(d['rerr1'][i])
    arr.append(d['rerr2'][i])
    arr.append(d['mp'][i])
    arr.append(d['merr1'][i])
    arr.append(d['merr2'][i])
    arr.append(d['p'][i])
    arr.append(d['rs'][i])
    arr.append(d['ms'][i])
    arr.append(d['disc_method'][i])
    arr.append(d['system_disc_method'][i])
    arr.append(d['temp'][i])
    arr.append(d['mag'][i])
    arr.append(d['instrument'][i])
    arr.append(d['ref'][i])
    myData.append(arr)
outputFile = open('C:/Users/djhoo/Documents/maiasaurus-master/catalogues/oviraptor_master.csv', 'w', newline='')
with outputFile:
    writer = csv.writer(outputFile)
    writer.writerows(myData)
print('Writing Complete!')

Writing Complete!
